In [1]:
# notebook last tested with abtem version 1.0.0beta32
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mimg

import cv2
import random

import numpy as np
import numpy.ma as ma
import pandas as pd
import shutil
import os

from scipy import constants
from scipy import misc
from scipy.ndimage import gaussian_filter

from abtem import __version__
from abtem import *

from ase.io import read
from ase.cluster import wulff_construction
from ase import Atoms
import ase.data



from PIL import Image
from skimage import data
from skimage import filters
from skimage import exposure
from skimage.filters import gaussian

/zhome/ab/7/153983/project/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Function that creates a custom sized circle mask at given h,w
def create_circular_mask(h, w, center=None, radius=None):
    if center is None: # use the middle of the image
        center = (int(w/2), int(h/2))
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((Y - center[0])**2 + (X-center[1])**2)

    mask = dist_from_center <= radius
    return mask

In [3]:
def apply_mask(img):
    image = Image.open(img)
    image_arr = np.array(image)

    if image_arr.shape[-1] == 4:
        image_arr = image_arr[:,:,:-1]

    image_arr = image_arr.astype(float)
    image_arr /= 255

    if len(image_arr.shape) == 3:
        weights = np.array([1, 1, 1])
        image_arr = np.dot(image_arr, weights)

    image_center = (300,400)
    image_center_arr = np.array(image_center)
    inner_mask = create_circular_mask(600 ,800 ,
                                      center = image_center_arr, radius = 100)
    visible_mask = create_circular_mask(600 ,800 ,
                                      center = image_center_arr, radius = 250)
    visible_mask = np.logical_xor(visible_mask, inner_mask)
    # Apply mask and return 
    
    return ma.masked_array(image_arr, ~visible_mask)

In [16]:
l = os.listdir('Database')
image_paths = []
for i in range(len(l)):
    if (".ipynb_checkpoints" not in l[i]) and (l[i] not in ["cleanImg", "train", "test", "valid"]):
        if i < 3000:
            directory = "train"
        elif i < 4000:
            directory = "test"
        else:
            directory = "valid"
        img = apply_mask("{}/{}".format("Database", l[i]))
        euler = l[i].split("_")
        phi1 = float(euler[1])
        PHI  = float(euler[2])
        phi2 = float(euler[3][:-4])
        image_path = "{}/{}/{}".format("Database", directory, l[i])
        image_paths.append([image_path, phi1, PHI, phi2])
        mimg.imsave(image_path, img, cmap = 'gray')
        shutil.move("Database/{}".format(l[i]), "Database/cleanImg/{}".format(l[i]))
df = pd.DataFrame(image_paths, columns =['ImagePath','phi1','PHI','phi2'])   

In [17]:
folders = ["train", "test", "valid"]
image_paths = []
for folder in folders:
    l = os.listdir('Database/{}'.format(folder))
    for image in l:
        euler = image.split("_")
        phi1 = float(euler[1])
        PHI  = float(euler[2])
        phi2 = float(euler[3][:-4])
        image_paths.append(['Database/{}/{}'.format(folder, image), phi1, PHI, phi2])

df = pd.DataFrame(image_paths, columns =['ImagePath','phi1','PHI','phi2'])    
df.to_csv('lables.csv')  